In [38]:
import nltk

[nltk_data] Downloading package treebank to
[nltk_data]     /home/ostapkharysh/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [28]:
print(nltk.bigrams)
print(nltk.trigrams)
print(nltk.ngrams)

<function bigrams at 0x7f6b84309268>
<function trigrams at 0x7f6b843092f0>
<function ngrams at 0x7f6b843091e0>


Text Analytics with Python.
A Practical Real-World Approach to Gaining Actionable Insights from your Data
Dipanjan Sarkar

In [30]:
from nltk.corpus import gutenberg
#from normalization import normalize_corpus
import nltk
from operator import itemgetter
# load corpus
alice = gutenberg.sents(fileids='carroll-alice.txt')
alice = [' '.join(ts) for ts in alice]
№norm_alice = filter(None, normalize_corpus(alice, lemmatize=False))

SyntaxError: invalid character in identifier (<ipython-input-30-8b292661e5a5>, line 8)

In [31]:
# print first line
print norm_alice[0]

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-31-2a7b28da4f16>, line 2)

In [ ]:
def flatten_corpus(corpus):
    return ' '.join([document.strip()for document in corpus])

In [ ]:
def compute_ngrams(sequence, n):
    return zip(*[sequence[index:] for index in range(n)])

In [ ]:
compute_ngrams([1,2,3,4], 2)

In [ ]:
compute_ngrams([1,2,3,4], 3)

In [ ]:
def get_top_ngrams(corpus, ngram_val=1, limit=5):
    corpus = flatten_corpus(corpus)
    tokens = nltk.word_tokenize(corpus)
    ngrams = compute_ngrams(tokens, ngram_val)
    ngrams_freq_dist = nltk.FreqDist(ngrams)
    sorted_ngrams_fd = sorted(ngrams_freq_dist.items(),     key=itemgetter(1), reverse=True)
    sorted_ngrams = sorted_ngrams_fd[0:limit]
    sorted_ngrams = [(' '.join(text), freq) for text, freq in sorted_ngrams]
    return sorted_ngrams

In [ ]:
# top 10 bigrams
get_top_ngrams(corpus=norm_alice, ngram_val=2, limit=10)

In [ ]:
get_top_ngrams(corpus=norm_alice, ngram_val=3, limit=10)

http://www.katrinerk.com/courses/python-worksheets/language-models-in-python

In [39]:
import nltk

from nltk.corpus import brown

# an nltk.FreqDist() is like a dictionary,
# but it is ordered by frequency.
# Also, nltk automatically fills the dictionary
# with counts when given a list of words.

freq_brown = nltk.FreqDist(brown.words())

list(freq_brown.keys())[:20]
freq_brown.most_common(20)

# an nltk.ConditionalFreqDist() counts frequencies of pairs.
# When given a list of bigrams, it maps each first word of a bigram
# to a FreqDist over the second words of the bigram.

cfreq_brown_2gram = nltk.ConditionalFreqDist(nltk.bigrams(brown.words()))

# conditions() in a ConditionalFreqDist are like keys()
# in a dictionary

cfreq_brown_2gram.conditions()

# the cfreq_brown_2gram entry for "my" is a FreqDist.

cfreq_brown_2gram["my"]

# here are the words that can follow after "my".
# We first access the FreqDist associated with "my",
# then the keys in that FreqDist

cfreq_brown_2gram["my"].keys()

# here are the 20 most frequent words to come after "my", with their frequencies

cfreq_brown_2gram["my"].most_common(20)

# an nltk.ConditionalProbDist() maps pairs to probabilities.
# One way in which we can do this is by using Maximum Likelihood Estimation (MLE)

cprob_brown_2gram = nltk.ConditionalProbDist(cfreq_brown_2gram, nltk.MLEProbDist)

# This again has conditions() wihch are like dictionary keys

cprob_brown_2gram.conditions()

# Here is what we find for "my": a Maximum Likelihood Estimation-based probability distribution,
# as a MLEProbDist object.

cprob_brown_2gram["my"]

# We can find the words that can come after "my" by using the function samples()

cprob_brown_2gram["my"].samples()

# Here is the probability of a particular pair:



cprob_brown_2gram["my"].prob("own")

#####

# We can also compute unigram probabilities (probabilities of individual words)

freq_brown_1gram = nltk.FreqDist(brown.words())

len_brown = len(brown.words())


def unigram_prob(word):

    return freq_brown_1gram[ word] / len_brown



#############

# The contents of cprob_brown_2gram, all these probabilities, now form a

# trained bigram language model. The typical use for a language model is

# to ask it for the probabillity of a word sequence

# P(how do you do) = P(how) * P(do|how) * P(you|do) * P(do | you)

prob_sentence = unigram_prob("how") * cprob_brown_2gram["how"].prob("do") * cprob_brown_2gram["do"].prob("you") * cprob_brown_2gram["you"].prob("do")

# result: 1.5639033871961e-09


###############

# We can also use a language model in another way:

# We can let it generate text at random

# This is not so useful, but can be insightful into what it is that

# the language model has been learning



cprob_brown_2gram["my"].generate()

# We can use this to generate text at random
# based on a given text of bigrams.
# Let's do this for the Sam "corpus"

corpus = """<s> I am Sam </s>
<s> Sam I am </s>
<s> I do not like green eggs and ham </s>"""

words = corpus.split()
cfreq_sam = nltk.ConditionalFreqDist(nltk.bigrams(words))
cprob_sam = nltk.ConditionalProbDist(cfreq_sam, nltk.MLEProbDist)

word = "<s>"
for index in range(50):
    word = cprob_sam[ word].generate()
    print(word, end = " ")

print("\n")

# Not a lot of variety. We need a bigger corpus.
# What kind of genres do we have in the Brown corpus?
brown.categories()

# Let's try Science Fiction.
cfreq_scifi = nltk.ConditionalFreqDist(nltk.bigrams(brown.words(categories = "science_fiction")))
cprob_scifi = nltk.ConditionalProbDist(cfreq_scifi, nltk.MLEProbDist)

word = "in"
for index in range(50):
    word = cprob_scifi[ word ].generate()
    print(word, end = " ")
print()

# try this with other Brown corpus categories.


# Here is how to do this with NLTK books:
import nltk
from nltk.book import *

def generate_text(text, initialword, numwords):
    bigrams = list(nltk.ngrams(text, 2))
    cpd = nltk.ConditionalProbDist(nltk.ConditionalFreqDist(bigrams), nltk.MLEProbDist)


word = initialword
for i in range(numwords):
    print(word, end = " ")
    word = cpd[ word].generate()
    print(word)

# Holy Grail
generate_text(text6, "I", 100)
# sense and sensibility
generate_text(text2, "I", 100)

I am Sam </s> <s> I do not like green eggs and ham </s> <s> Sam </s> <s> I do not like green eggs and ham </s> <s> I am Sam I am </s> <s> Sam </s> <s> I am </s> <s> Sam </s> <s> I do not like green 

a thousand Siddo . Gradually , as you have '' ? `` Ekstrohm said , but they would join the skiff hove into a nestling , and instrumental . The half-man lay back . Any little hand is a creepy feeling to see stars in lightyears or future , it *** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


NameError: name 'initialword' is not defined

In [34]:
# Katrin Erk Oct 07
# Updated Feb 11
#
# Word bigrams are just pairs of words.
# In the sentence "I went to the beach"
# the bigrams are:
#    I went
#    went to
#    to the
#    the beach
#
# Having counts of English bigrams from a very large text corpus
# can be useful for a number of purposes.
#
# for example for spelling correction:
# If I had mistyped the sentence as "I went to beach"
# then I might be able to find the error by seeing that
# the bigram "to beach" has a very low count, and
# "to the", "to a", and "the beach" have much larger counts.
#
# This program counts all word bigrams in a given text file
#
# usage:
# python3 count_bigrams.py <filename>
#
# <filename> is a text file.

import string
import sys

# complain if we didn't get a filename
# as a command line argument
if len(sys.argv) < 2:
    print("Please enter the name of a corpus file as a command line argument.")
    sys.exit()
   
# try opening file
# If the file doesn't exist, catch the error
try:
    f = open(sys.argv[1])
except IOError:
    print("Sorry, I could not find the file", sys.argv[1])
    print("Please try again.")
    sys.exit()
   
# read the contents of the whole file into ''filecontents''
filecontents = f.read()
       
# count bigrams
bigrams = {}
words_punct = filecontents.split()
# strip all punctuation at the beginning and end of words, and
# convert all words to lowercase.
# The following is a Python list comprehension. It is a command that transforms a list,
# here words_punct, into another list.
words = [ w.strip(string.punctuation).lower() for w in words_punct ]

# add special START, END tokens
words = ["START"] + words + ["END"]

for index, word in enumerate(words):
    if index < len(words) - 1:
        # we only look at indices up to the
        # next-to-last word, as this is
        # the last one at which a bigram starts
        w1 = words[index]
        w2 = words[index + 1]
        # bigram is a tuple,
        # like a list, but fixed.
        # Tuples can be keys in a dictionary
        bigram = (w1, w2)

        if bigram in bigrams:
            bigrams[ bigram ] = bigrams[ bigram ] + 1
        else:
            bigrams[ bigram ] = 1
        # or, more simply, like this:
        # bigrams[bigram] = bigrams.get(bigram, 0) + 1

# sort bigrams by their counts
sorted_bigrams = sorted(bigrams.items(), key = lambda pair:pair[1], reverse = True)

for bigram, count in sorted_bigrams
    print(bigram, ":", count)

SyntaxError: invalid syntax (<ipython-input-34-4201941e78f9>, line 83)

Bi-gram extraction and Language model

https://appliedmachinelearning.blog/2017/04/30/language-identification-from-texts-using-bi-gram-model-pythonnltk/

In [ ]:
from nltk.collocations import BigramCollocationFinder
import re
import codecs
import numpy as np
import string

def train_language(path,lang_name):
    words_all = []
    translate_table = dict((ord(char), None) for char in string.punctuation)
    # reading the file in unicode format using codecs library    
    with codecs.open(path,"r","utf-8") as filep:
         
        for i,line in enumerate(filep):            
            # extracting the text sentence from each line         
            line = " ".join(line.split()[1:])
            line = line.lower()   # to lower case
            line = re.sub(r"\d+", "", line) # remove digits
            
            if len(line) != 0:
                line = line.translate(translate_table) # remove punctuations
                words_all += line
                words_all.append(" ") # append sentences with space
               
    all_str = ''.join(words_all)
    all_str = re.sub(' +',' ',all_str) # replace series of spaces with single space
    seq_all = [i for i in all_str]
    
    # extracting the bi-grams and sorting them according to their frequencies
    finder = BigramCollocationFinder.from_words(seq_all)
    finder.apply_freq_filter(5)
    bigram_model = finder.ngram_fd.viewitems()
    bigram_model = sorted(finder.ngram_fd.viewitems(), key=lambda item: item[1],reverse=True)  
    
    print bigram_model   
    np.save(lang_name+".npy",bigram_model) # save language model

if __name__ == "__main__":
    root = "train\\"
    lang_name = ["french","english","german","italian","dutch","spanish"]
    train_lang_path = ["fra_news_2010_30K-text\\fra_news_2010_30K-sentences.txt","eng_news_2015_30K\\eng_news_2015_30K-sentences.txt","deu_news_2015_30K\\deu_news_2015_30K-sentences.txt","ita_news_2010_30K-text\\ita_news_2010_30K-sentences.txt","nld_wikipedia_2016_30K\\nld_wikipedia_2016_30K-sentences.txt","spa_news_2011_30K\\spa_news_2011_30K-sentences.txt"]
    for i,p in enumerate(train_lang_path):
        train_language(root+p,lang_name[i])

Python/gensim: Creating bigrams

https://markhneedham.com/blog/2015/02/12/pythongensim-creating-bigrams-over-how-i-met-your-mother-transcripts/

In [ ]:
import nltk
import csv
import string
from gensim.models import Phrases
from gensim.models import Word2Vec
from nltk.corpus import stopwords

sentences = []
bigram = Phrases()
with open("data/import/sentences.csv", "r") as sentencesfile:
    reader = csv.reader(sentencesfile, delimiter = ",")
    reader.next()
    for row in reader:
        sentence = [word.decode("utf-8")
                    for word in nltk.word_tokenize(row[4].lower())
                    if word not in string.punctuation]
        sentences.append(sentence)
        bigram.add_vocab([sentence])

In [ ]:
list(bigram[sentences])[:5]

In [ ]:
bigram_counter = Counter()
for key in bigram.vocab.keys():
    if key not in stopwords.words("english"):
        if len(key.split("_")) > 1:
            bigram_counter[key] += bigram.vocab[key]

for key, counts in bigram_counter.most_common(20):
    print '{0: <20} {1}'.format(key.encode("utf-8"), counts)

In [ ]:
Kneser Ney smoo

In [ ]:
https://stackoverflow.com/questions/35242155/kneser-ney-smoothing-of-trigrams-using-python-nltk

In [ ]:
from nltk.util import ngrams
    from nltk.corpus import gutenberg

    gut_ngrams = ( ngram for sent in gutenberg.sents() for ngram in ngrams(sent, 3, pad_left = True, pad_right = True, right_pad_symbol='EOS', left_pad_symbol="BOS"))
    freq_dist = nltk.FreqDist(gut_ngrams)
    kneser_ney = nltk.KneserNeyProbDist(freq_dist)

    prob_sum = 0
    for i in kneser_ney.samples():
        if i[0] == "I" and i[1] == "confess":
            prob_sum += kneser_ney.prob(i)
            print "{0}:{1}".format(i, kneser_ney.prob(i))
    print prob_sum

Завдання 1. Впорядкувати, перевірити notepad.
Завдання 2. Вивчити засоби бібліотеки gensim по створенню n-gram моделей мови https://radimrehurek.com/gensim/models/phrases.html 
Завдання 3. N-грам моделі для української мови. Література, моделі, власні моделі (Вікіпедія).